In [1]:
import pandas as pd


def calculate_demographic_data(print_data=True):
    """
    Demographic Data Analyzer (freeCodeCamp)
    Lê adult.data (CSV sem cabeçalho) e calcula métricas com Pandas.
    """

    # Nomes das colunas (dataset não tem cabeçalho)
    columns = [
        "age","workclass","fnlwgt","education","education-num",
        "marital-status","occupation","relationship","race","sex",
        "capital-gain","capital-loss","hours-per-week","native-country","salary"
    ]

    # Leitura do arquivo adult.data
    df = pd.read_csv(
        "adult.data",           # use "adult.data.csv" se renomear o arquivo
        header=None,            # porque não tem cabeçalho
        names=columns,          # define os nomes das colunas
        na_values=["?"],        # trata valores faltantes
        skipinitialspace=True   # remove espaços depois da vírgula
    )

    # Remover espaços extras em todas as colunas de texto
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].str.strip()

    # 1) Número de pessoas por raça
    race_count = df["race"].value_counts()

    # 2) Idade média dos homens
    average_age_men = round(df.loc[df["sex"] == "Male", "age"].mean(), 1)

    # 3) % de pessoas com Bachelor's
    percentage_bachelors = round((df["education"].eq("Bachelors").mean() * 100), 1)

    # 4) % com educação avançada (Bachelors, Masters, Doctorate) que ganham >50K
    higher_edu_mask = df["education"].isin(["Bachelors", "Masters", "Doctorate"])
    higher_education = df[higher_edu_mask]
    higher_education_rich = round(
        (higher_education["salary"].eq(">50K").mean() * 100), 1
    )

    # 5) % sem educação avançada que ganham >50K
    lower_education = df[~higher_edu_mask]
    lower_education_rich = round(
        (lower_education["salary"].eq(">50K").mean() * 100), 1
    )

    # 6) Mínimo de horas por semana
    min_work_hours = int(df["hours-per-week"].min())

    # 7) % de pessoas que trabalham o mínimo e ganham >50K
    min_workers = df[df["hours-per-week"] == min_work_hours]
    rich_percentage = round((min_workers["salary"].eq(">50K").mean() * 100), 1) if not min_workers.empty else 0.0

    # 8) País com maior % de >50K
    salary_by_country = (
        df.groupby("native-country")["salary"]
        .value_counts(normalize=True)
        .mul(100)
        .unstack(fill_value=0)
    )
    highest_earning_country = salary_by_country[">50K"].idxmax()
    highest_earning_country_percentage = round(salary_by_country[">50K"].max(), 1)

    # 9) Ocupação mais popular entre >50K na Índia
    india_rich = df[(df["native-country"] == "India") & (df["salary"] == ">50K")]
    top_IN_occupation = india_rich["occupation"].value_counts().idxmax() if not india_rich.empty else None

    if print_data:
        print("Number of each race:\n", race_count)
        print("Average age of men:", average_age_men)
        print(f"Percentage with Bachelors degrees: {percentage_bachelors}%")
        print(f"Percentage with higher education that earn >50K: {higher_education_rich}%")
        print(f"Percentage without higher education that earn >50K: {lower_education_rich}%")
        print(f"Min work time: {min_work_hours} hours/week")
        print(f"Percentage of rich among those who work fewest hours: {rich_percentage}%")
        print("Country with highest percentage of rich:", highest_earning_country, f"({highest_earning_country_percentage}%)")
        print("Top occupations in India:", top_IN_occupation)

    return {
        "race_count": race_count,
        "average_age_men": average_age_men,
        "percentage_bachelors": percentage_bachelors,
        "higher_education_rich": higher_education_rich,
        "lower_education_rich": lower_education_rich,
        "min_work_hours": min_work_hours,
        "rich_percentage": rich_percentage,
        "highest_earning_country": highest_earning_country,
        "highest_earning_country_percentage": highest_earning_country_percentage,
        "top_IN_occupation": top_IN_occupation,
    }


if __name__ == "__main__":
    calculate_demographic_data(print_data=True)


Number of each race:
 race
White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: count, dtype: int64
Average age of men: 39.4
Percentage with Bachelors degrees: 16.4%
Percentage with higher education that earn >50K: 46.5%
Percentage without higher education that earn >50K: 17.4%
Min work time: 1 hours/week
Percentage of rich among those who work fewest hours: 10.0%
Country with highest percentage of rich: Iran (41.9%)
Top occupations in India: Prof-specialty
